Before running any of the following code, make sure you import the api dataset with `mongoimport --type json -d la_crime_data -c data_2020_2023 --drop --jsonArray jsondata_from_api.json`

In [446]:
# Import dependencies
from pathlib import Path
import pandas as pd
from pymongo import MongoClient
from pprint import pprint

In [447]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [448]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruitsDB', 'gardenDB', 'la_crime_data', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [449]:
# assign the la_crime_data database to a variable name
db = mongo['la_crime_data']

In [450]:
# review the collections in our new database
print(db.list_collection_names())

['data_2021', 'data_2023', 'data_2020_2023', 'data_2020', 'data_2022']


In [451]:
# review a document in the artefacts collection
pprint(db.data_2020_2023.find_one())

{'_id': ObjectId('658173c9f582a99749c5bd04'),
 'area': '03',
 'area_name': 'Southwest',
 'crm_cd': '624',
 'crm_cd_1': '624',
 'crm_cd_desc': 'BATTERY - SIMPLE ASSAULT',
 'date_occ': '2020-01-08T00:00:00.000',
 'date_rptd': '2020-01-08T00:00:00.000',
 'dr_no': '010304468',
 'lat': '34.0141',
 'location': '1100 W  39TH                         PL',
 'lon': '-118.2978',
 'mocodes': '0444 0913',
 'part_1_2': '2',
 'premis_cd': '501',
 'premis_desc': 'SINGLE FAMILY DWELLING',
 'rpt_dist_no': '0377',
 'status': 'AO',
 'status_desc': 'Adult Other',
 'time_occ': '2230',
 'vict_age': '36',
 'vict_descent': 'B',
 'vict_sex': 'F',
 'weapon_desc': 'STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)',
 'weapon_used_cd': '400'}


In [452]:
# assign the collection to a variable
artefacts = db['data_2020_2023']

In [453]:
# Count the number of documents in the collection
document_count = artefacts.count_documents({})

print(f"Total number of documents: {document_count}")

Total number of documents: 857658


In [454]:
# Show only columns needed
query = {'lat': {'$ne': "0"}}
fields = {"dr_no": 1,"date_rptd": 1, "date_occ": 1, "time_occ": 1, "area_name": 1, "crm_cd": 1, "crm_cd_desc": 1, "crm_cd_desc": 1, "premis_desc": 1,"location": 1,"cross_street": 1,"lat": 1,"lon": 1,"vict_age": 1,"vict_sex": 1,"vict_descent":1}
sort = [("dr_no", 1)]
limit = 5

# Cast the results as a list and save the results to a variable
results = list(artefacts.find(query, fields).sort(sort))

# Pretty print the results
# pprint(results)

In [455]:
# Create a DataFrame from the results list
df = pd.DataFrame(results)

# Display the DataFrame
print(df.head())

                        _id      dr_no                date_rptd  \
0  658173c9f582a99749c5bd04  010304468  2020-01-08T00:00:00.000   
1  658173ccf582a99749c5fb63       0817  2020-09-20T00:00:00.000   
2  658173c9f582a99749c5bd05  190101086  2020-01-02T00:00:00.000   
3  658173d5f582a99749c6c32a  190101087  2020-01-02T00:00:00.000   
4  658173cff582a99749c636b3  190326475  2020-03-01T00:00:00.000   

                  date_occ time_occ   area_name crm_cd  \
0  2020-01-08T00:00:00.000     2230   Southwest    624   
1  2020-09-19T00:00:00.000     1700  Devonshire    510   
2  2020-01-01T00:00:00.000     0330     Central    624   
3  2020-01-01T00:00:00.000     0510     Central    626   
4  2020-03-01T00:00:00.000     2130    Wilshire    510   

                         crm_cd_desc vict_age vict_sex vict_descent  \
0           BATTERY - SIMPLE ASSAULT       36        F            B   
1                   VEHICLE - STOLEN        0      NaN          NaN   
2           BATTERY - SIMPLE ASSAUL

In [456]:
#check coordinates
maxlon = (df["lon"].max())
maxlon

'-118.6676'

In [457]:
#check coordinates
minlon = (df["lon"].min())
minlon

'-118.1554'

In [458]:
#check coordinates
maxlat = (df["lat"].max())
maxlat

'34.3343'

In [459]:
#check coordinates
minlat = (df["lat"].min())
minlat

'33.7061'

In [460]:
#count records
row_count = len(df)
print("Number of records in the DataFrame:", row_count)

Number of records in the DataFrame: 855395


In [461]:
df.columns

Index(['_id', 'dr_no', 'date_rptd', 'date_occ', 'time_occ', 'area_name',
       'crm_cd', 'crm_cd_desc', 'vict_age', 'vict_sex', 'vict_descent',
       'premis_desc', 'location', 'lat', 'lon', 'cross_street'],
      dtype='object')

In [462]:
#rename columns
renamed_df = df.rename(columns={"dr_no": "division_number",
                                        "date_rptd": "date_reported",
                                        "date_occ": "date_occured",
                                        "time_occ": "time_occured",
                                        "crm_cd_desc": "crime_description",
                                        "vict_age": "victim_age",
                                        "vict_sex": "victim_sex",
                                        "vict_descent": "victim_descent" })
renamed_df.head()

,_id,division_number,date_reported,date_occured,time_occured,area_name,crm_cd,crime_description,victim_age,victim_sex,victim_descent,premis_desc,location,lat,lon,cross_street
0,658173c9f582a99749c5bd04,010304468,2020-01-08T00:00:00.000,2020-01-08T00:00:00.000,2230,Southwest,624,BATTERY - SIMPLE ASSAULT,36,F,B,SINGLE FAMILY DWELLING,1100 W 39TH PL,34.0141,-118.2978,NaN
1,658173ccf582a99749c5fb63,0817,2020-09-20T00:00:00.000,2020-09-19T00:00:00.000,1700,Devonshire,510,VEHICLE - STOLEN,0,NaN,NaN,STREET,9100 RUBIO AV,34.2367,-118.4955,NaN
2,658173c9f582a99749c5bd05,190101086,2020-01-02T00:00:00.000,2020-01-01T00:00:00.000,0330,Central,624,BATTERY - SIMPLE ASSAULT,25,M,H,SIDEWALK,700 S HILL ST,34.0459,-118.2545,NaN
3,658173d5f582a99749c6c32a,190101087,2020-01-02T00:00:00.000,2020-01-01T00:00:00.000,0510,Central,626,INTIMATE PARTNER - SIMPLE ASSAULT,53,F,B,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",300 E 5TH ST,34.0447,-118.2452,NaN
4,658173cff582a99749c636b3,190326475,2020-03-01T00:00:00.000,2020-03-01T00:00:00.000,2130,Wilshire,510,VEHICLE - STOLEN,0,M,O,STREET,1900 S LONGWOOD AV,34.0375,-118.3506,NaN


In [463]:
# order and removed id column in df
cleaned_df = renamed_df[["division_number", "date_reported", "date_occured", "time_occured", "area_name", "crm_cd","crime_description","victim_age",
                            "victim_sex","victim_descent","premis_desc","location","cross_street","lat","lon"]]

cleaned_df.tail()

,division_number,date_reported,date_occured,time_occured,area_name,crm_cd,crime_description,victim_age,victim_sex,victim_descent,premis_desc,location,cross_street,lat,lon
855390,239906039,2023-01-26T00:00:00.000,2023-01-26T00:00:00.000,1510,West Valley,510,VEHICLE - STOLEN,0,M,H,STREET,7700 LINDLEY AV,NaN,34.2104,-118.5273
855391,239909037,2023-03-03T00:00:00.000,2023-03-02T00:00:00.000,2000,Newton,510,VEHICLE - STOLEN,0,M,W,STREET,BROADWAY,61ST ST,33.9842,-118.2783
855392,239909747,2023-03-12T00:00:00.000,2023-03-12T00:00:00.000,1500,Rampart,626,INTIMATE PARTNER - SIMPLE ASSAULT,30,M,B,OTHER PREMISE,1400 W 6TH ST,NaN,34.0563,-118.2673
855393,239916487,2023-06-04T00:00:00.000,2023-06-04T00:00:00.000,1930,77th Street,510,VEHICLE - STOLEN,0,X,X,STREET,62ND ST,HOOVER ST,33.9837,-118.2871
855394,239930556,2023-11-11T00:00:00.000,2023-11-11T00:00:00.000,2030,Hollenbeck,890,FAILURE TO YIELD,0,X,X,STREET,HUNTINGTON DR,SOTO ST,34.0802,-118.1848


In [464]:
# format df to correct datatypes
cleaned_df.dtypes

division_number      object
date_reported        object
date_occured         object
time_occured         object
area_name            object
crm_cd               object
crime_description    object
victim_age           object
victim_sex           object
victim_descent       object
premis_desc          object
location             object
cross_street         object
lat                  object
lon                  object
dtype: object

In [465]:
#update columns to correct date data types
cleaned_df['date_reported'] = pd.to_datetime(cleaned_df['date_reported']) 
cleaned_df['date_occured'] = pd.to_datetime(cleaned_df['date_occured'])

C:\Users\michaelz\AppData\Local\Temp\ipykernel_40888\1398910109.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['date_reported'] = pd.to_datetime(cleaned_df['date_reported'])
C:\Users\michaelz\AppData\Local\Temp\ipykernel_40888\1398910109.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['date_occured'] = pd.to_datetime(cleaned_df['date_occured'])


In [466]:
#update columns to correct datatypes
cleaned_df = cleaned_df.astype({"division_number": int}, errors='raise')
cleaned_df = cleaned_df.astype({"time_occured": int}, errors='raise')
cleaned_df = cleaned_df.astype({"crm_cd": int}, errors='raise')
cleaned_df = cleaned_df.astype({"victim_age": int}, errors='raise')
cleaned_df = cleaned_df.astype({"lat": float}, errors='raise')
cleaned_df = cleaned_df.astype({"lon": float}, errors='raise')

In [467]:
cleaned_df.head(10)

,division_number,date_reported,date_occured,time_occured,area_name,crm_cd,crime_description,victim_age,victim_sex,victim_descent,premis_desc,location,cross_street,lat,lon
0,10304468,2020-01-08,2020-01-08,2230,Southwest,624,BATTERY - SIMPLE ASSAULT,36,F,B,SINGLE FAMILY DWELLING,1100 W 39TH PL,NaN,34.0141,-118.2978
1,817,2020-09-20,2020-09-19,1700,Devonshire,510,VEHICLE - STOLEN,0,NaN,NaN,STREET,9100 RUBIO AV,NaN,34.2367,-118.4955
2,190101086,2020-01-02,2020-01-01,330,Central,624,BATTERY - SIMPLE ASSAULT,25,M,H,SIDEWALK,700 S HILL ST,NaN,34.0459,-118.2545
3,190101087,2020-01-02,2020-01-01,510,Central,626,INTIMATE PARTNER - SIMPLE ASSAULT,53,F,B,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",300 E 5TH ST,NaN,34.0447,-118.2452
4,190326475,2020-03-01,2020-03-01,2130,Wilshire,510,VEHICLE - STOLEN,0,M,O,STREET,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
5,191501505,2020-01-01,2020-01-01,1730,N Hollywood,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",5400 CORTEEN PL,NaN,34.1685,-118.4019
6,191921269,2020-01-01,2020-01-01,415,Mission,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",31,X,X,BEAUTY SUPPLY STORE,14400 TITUS ST,NaN,34.2198,-118.4468
7,200100001,2020-02-26,2020-02-25,2000,Central,510,VEHICLE - STOLEN,0,NaN,NaN,PARKING LOT,500 N FIGUEROA ST,NaN,34.0617,-118.2469
8,200100002,2020-08-15,2020-08-14,1740,Central,510,VEHICLE - STOLEN,0,NaN,NaN,STREET,200 E 9TH ST,NaN,34.0404,-118.2504
9,200100003,2020-08-15,2020-08-14,2200,Central,510,VEHICLE - STOLEN,0,NaN,NaN,STREET,800 S HILL ST,NaN,34.0462,-118.2585


In [468]:
# format df to correct datatypes
cleaned_df.dtypes

division_number               int32
date_reported        datetime64[ns]
date_occured         datetime64[ns]
time_occured                  int32
area_name                    object
crm_cd                        int32
crime_description            object
victim_age                    int32
victim_sex                   object
victim_descent               object
premis_desc                  object
location                     object
cross_street                 object
lat                         float64
lon                         float64
dtype: object

In [469]:
#clean victim_age
df_new = cleaned_df.drop(df[cleaned_df["victim_age"] < 0].index)
print(df_new)

        division_number date_reported date_occured  time_occured    area_name  \
0              10304468    2020-01-08   2020-01-08          2230    Southwest   
1                   817    2020-09-20   2020-09-19          1700   Devonshire   
2             190101086    2020-01-02   2020-01-01           330      Central   
3             190101087    2020-01-02   2020-01-01           510      Central   
4             190326475    2020-03-01   2020-03-01          2130     Wilshire   
...                 ...           ...          ...           ...          ...   
855390        239906039    2023-01-26   2023-01-26          1510  West Valley   
855391        239909037    2023-03-03   2023-03-02          2000       Newton   
855392        239909747    2023-03-12   2023-03-12          1500      Rampart   
855393        239916487    2023-06-04   2023-06-04          1930  77th Street   
855394        239930556    2023-11-11   2023-11-11          2030   Hollenbeck   

        crm_cd             

In [470]:
#clean victim_age
unique_age = df_new["victim_age"].unique()
print(unique_age)

[ 36   0  25  53  76  31  23  61  29  35  41  24  51  18  66  27  62  39
  46  71  40  19   7  69   2  26  52  38  55  22  12  28  54  37  42  50
  44  49  17  60  34  45  79  48  33  32  59  68  57  47  30  56  63  58
  20  21  16  70  43  67  64  75  13  74  65  14  72   3  90  85  81   8
  94  73   5  78  15  80  99  97  86  83   6  84  10  82  77  88  89  11
  95   9  96  87  93   4  92  91  98 120]


In [471]:
# check unique sex
unique_sex = df_new["victim_sex"].unique()
print(unique_sex)

['F' nan 'M' 'X' 'H' '-']


In [472]:
#clean victim_sex
df_new = df_new.drop(df_new[(df_new['victim_sex'] == "-") | (df_new["victim_sex"] == "H")].index)

In [473]:
df_new['victim_sex'] = df_new['victim_sex'] .replace(["F", "M", "X"], ["Female", "Male", "Unknown"])

In [474]:
# check unique sex again
unique_sex = df_new["victim_sex"].unique()
print(unique_sex)

['Female' nan 'Male' 'Unknown']


In [475]:
# Wassim's code - based on refrence below
# UCR Reporting Crime Codes
# Reference: "UCR-COMPSTAT062618.pdf"

# Homicide
homicide_codes = [110, 113]

# Rape
rape_codes = [121, 122, 815, 820, 821]

# Robbery
robbery_codes = [210, 220]

# Aggravated Assault
agg_assault_codes = [230, 231, 235]

# Domestic Violence
domestic_violence_codes = [236, 250, 251, 761, 926,
                           626, 627, 647, 763, 928, 930]

# Simple Assault
simple_assault_codes = [435, 436, 437, 622, 623, 624, 625]

# Burglary
burglary_codes = [310, 320]

# Grand Theft Auto (Motor Vehicle Theft)
gta_codes = [510, 520, 433]

# Burglary or Theft from Vehicle
btfv_codes = [330, 331, 410, 420, 421]

# Personal Theft
personal_theft_codes = [350, 351, 352, 353, 450, 451, 452, 453]

# Other Theft
other_theft_codes = [341, 343, 345, 440, 441, 442, 443, 444, 445,
                     470, 471, 472, 473, 474, 475, 480, 485, 487, 491]

ucr_dict = {"Homicide": homicide_codes,
            "Rape": rape_codes,
            "Robbery": robbery_codes,
            "Aggravated Assault": agg_assault_codes,
            "Domestic Violence": domestic_violence_codes,
            "Simple Assault": simple_assault_codes,
            "Burglary": burglary_codes,
            "Grand Theft Auto": gta_codes,
            "Burglary/Theft from Vehicle": btfv_codes,
            "Personal Theft": personal_theft_codes,
            "Other Theft": other_theft_codes}

# Custom map function to insert the crime category for each datapoint in the "crime_category" column based on 'crm_cd'
# Reference: https://favtutor.com/blogs/pandas-map#:~:text=The%20map%20function%20is%20one,DataFrame%20with%20the%20modified%20values.
def map_crime_categories(code):
    # For every crime key and its corresponding list of code values...
    # If code from current crime_df row is in the list of codes from current crime key...
    # Append the "crime_category" column for current row with current crime key
    # Otherwise, Append the "crime_category" column for current row with "Other"
    for crime, codes in ucr_dict.items():
        if code in codes:
            return crime
    return "Other"


df_new["crime_category"] = df_new["crm_cd"].map(map_crime_categories)

df_new.head()

,division_number,date_reported,date_occured,time_occured,area_name,crm_cd,crime_description,victim_age,victim_sex,victim_descent,premis_desc,location,cross_street,lat,lon,crime_category
0,10304468,2020-01-08,2020-01-08,2230,Southwest,624,BATTERY - SIMPLE ASSAULT,36,Female,B,SINGLE FAMILY DWELLING,1100 W 39TH PL,NaN,34.0141,-118.2978,Simple Assault
1,817,2020-09-20,2020-09-19,1700,Devonshire,510,VEHICLE - STOLEN,0,NaN,NaN,STREET,9100 RUBIO AV,NaN,34.2367,-118.4955,Grand Theft Auto
2,190101086,2020-01-02,2020-01-01,330,Central,624,BATTERY - SIMPLE ASSAULT,25,Male,H,SIDEWALK,700 S HILL ST,NaN,34.0459,-118.2545,Simple Assault
3,190101087,2020-01-02,2020-01-01,510,Central,626,INTIMATE PARTNER - SIMPLE ASSAULT,53,Female,B,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",300 E 5TH ST,NaN,34.0447,-118.2452,Domestic Violence
4,190326475,2020-03-01,2020-03-01,2130,Wilshire,510,VEHICLE - STOLEN,0,Male,O,STREET,1900 S LONGWOOD AV,NaN,34.0375,-118.3506,Grand Theft Auto


In [476]:
# Create a reference to each dataset. 
descent_csv = Path("la_crime_descent.csv")

In [477]:
# Read in dataset as a DataFrame.
descent_df = pd.read_csv(descent_csv)

In [478]:
# Merge the two DataFrames together based on the descent code they share
df_new_clean = pd.merge(df_new, descent_df, on=["victim_descent"])
df_new_clean.head()

,division_number,date_reported,date_occured,time_occured,area_name,crm_cd,crime_description,victim_age,victim_sex,victim_descent,premis_desc,location,cross_street,lat,lon,crime_category,victim_descent_name
0,10304468,2020-01-08,2020-01-08,2230,Southwest,624,BATTERY - SIMPLE ASSAULT,36,Female,B,SINGLE FAMILY DWELLING,1100 W 39TH PL,NaN,34.0141,-118.2978,Simple Assault,Black
1,190101087,2020-01-02,2020-01-01,510,Central,626,INTIMATE PARTNER - SIMPLE ASSAULT,53,Female,B,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",300 E 5TH ST,NaN,34.0447,-118.2452,Domestic Violence,Black
2,200100507,2020-01-04,2020-01-04,200,Central,341,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",23,Male,B,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",700 BERNARD ST,NaN,34.0677,-118.2398,Other Theft,Black
3,200100528,2020-01-11,2020-01-10,2015,Central,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",41,Male,B,UNDERPASS/BRIDGE*,FIGUEROA,TEMPLE,34.0606,-118.2477,Aggravated Assault,Black
4,200100535,2020-01-14,2020-01-14,1330,Central,210,ROBBERY,66,Male,B,ALLEY,7TH,HILL,34.0463,-118.2550,Robbery,Black


In [479]:
df_new_clean = df_new_clean[["division_number","date_reported", "date_occured", "time_occured", "area_name", "crime_category","crime_description","victim_age",
                            "victim_sex","victim_descent_name","premis_desc","location","cross_street","lat","lon"]]

df_new_clean.tail()

,division_number,date_reported,date_occured,time_occured,area_name,crime_category,crime_description,victim_age,victim_sex,victim_descent_name,premis_desc,location,cross_street,lat,lon
742457,231304277,2023-01-04,2023-01-03,752,Newton,Other,THEFT OF IDENTITY,46,Female,Laotian,CYBERSPACE,5400 FORTUNA ST,NaN,33.9939,-118.2456
742458,231305642,2023-02-02,2023-02-02,100,Newton,Other,THEFT OF IDENTITY,22,Female,Laotian,CYBERSPACE,1100 E 54TH ST,NaN,33.9939,-118.2565
742459,231517913,2023-11-21,2023-07-24,2235,N Hollywood,Other,THEFT OF IDENTITY,51,Female,Laotian,SINGLE FAMILY DWELLING,12700 BARBARA ANN ST,NaN,34.2002,-118.4116
742460,231806444,2023-02-15,2023-02-02,1,Southeast,Other,THEFT OF IDENTITY,55,Female,Laotian,SINGLE FAMILY DWELLING,700 E 105TH ST,NaN,33.9410,-118.2630
742461,232012253,2023-06-29,2023-06-29,521,Olympic,Other,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",43,Female,Laotian,PARKING LOT,500 N SERRANO AV,NaN,34.0799,-118.3069


In [480]:
df_new_clean.dtypes

division_number                 int32
date_reported          datetime64[ns]
date_occured           datetime64[ns]
time_occured                    int32
area_name                      object
crime_category                 object
crime_description              object
victim_age                      int32
victim_sex                     object
victim_descent_name            object
premis_desc                    object
location                       object
cross_street                   object
lat                           float64
lon                           float64
dtype: object

In [481]:
#count records
row_count = len(df_new_clean)
print("Number of records in the DataFrame:", row_count)

Number of records in the DataFrame: 742462


In [485]:
#check for duplicates
duplicate_rows = df_new_clean.duplicated(subset=['division_number'])

# print duplicate rows
print(duplicate_rows)

0         False
1         False
2         False
3         False
4         False
          ...  
742457    False
742458    False
742459    False
742460    False
742461    False
Length: 742462, dtype: bool


In [488]:
df_new_clean = df_new_clean.drop_duplicates(subset='division_number', keep="first")

In [489]:
#count records after duplicates
row_count = len(df_new_clean)
print("Number of records in the DataFrame:", row_count)

Number of records in the DataFrame: 739334


In [490]:
# save to csv
df_new_clean.to_csv('la_data_full_2020_2023.csv', index=False)

In [406]:
df_sample_clean = df_new_clean.tail(1000)

In [407]:
df_sample_clean.to_csv('la_data_2020_2023_sample.csv', index=False)